In [11]:
from fishsense_api_sdk.client import Client
from sklearn.cluster import HDBSCAN

In [12]:
URL = "http://localhost:8000"
DIVE_ID = 279

HDBSCAN_MIN_SAMPLES=2

In [13]:
client = Client(URL)

In [14]:
dive = await client.dives.get(dive_id=DIVE_ID)

dive

Dive(id=279, name='111323_Alligator DeepEast Drift2_FSL05', path='drive-download-20240307T1050Z/112023_Alligator/111323_Alligator/111323_Alligator_FSL05/111323_Alligator DeepEast Drift2_FSL05', dive_datetime=datetime.datetime(2023, 11, 13, 12, 34, 8, tzinfo=TzInfo(0)), priority=<Priority.HIGH: 'HIGH'>, flip_dive_slate=None, camera_id=5, dive_slate_id=None)

In [15]:
images = await client.images.get(dive_id=dive.id)

len(images), images[0]

(155,
 Image(id=78666, path='drive-download-20240307T1050Z/112023_Alligator/111323_Alligator/111323_Alligator_FSL05/111323_Alligator DeepEast Drift2_FSL05/PB130168.ORF', taken_datetime=datetime.datetime(2023, 11, 13, 12, 3, 31, tzinfo=TzInfo(0)), checksum='7252dfb0db0b855616e45607f58c1c93', is_canonical=True, dive_id=279, camera_id=5))

In [16]:
timestamps = [
    img.taken_datetime.timestamp() if img.taken_datetime else 0
    for img in images
]

len(timestamps), timestamps[0]

(155, 1699877011.0)

In [17]:
X = [[ts] for ts in timestamps]

In [18]:
db = HDBSCAN(min_cluster_size=HDBSCAN_MIN_SAMPLES).fit(X)
labels = db.labels_

len(labels), len(set(labels)), labels[0]

(155, 52, np.int64(4))

In [19]:
clusters = {}
cluster_len = len(set(labels)) - (1 if -1 in labels else 0)
for label, img in zip(labels, images):
    if label == -1:
        label = cluster_len
        cluster_len += 1

    if label not in clusters:
        clusters[label] = []
    clusters[label].append(img)

clusters = list(clusters.values())

len(clusters), len(clusters[0]), clusters[0][0]

(53,
 2,
 Image(id=78666, path='drive-download-20240307T1050Z/112023_Alligator/111323_Alligator/111323_Alligator_FSL05/111323_Alligator DeepEast Drift2_FSL05/PB130168.ORF', taken_datetime=datetime.datetime(2023, 11, 13, 12, 3, 31, tzinfo=TzInfo(0)), checksum='7252dfb0db0b855616e45607f58c1c93', is_canonical=True, dive_id=279, camera_id=5))

In [20]:
for cluster in clusters:
    image_ids = [img.id for img in cluster]

    await client.images.post_cluster(DIVE_ID, image_ids)